In [1]:
import pandas as pd
import csv
import os
import numpy as np
import shutil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm 
#from keras.utils import to_categorical
import cv2
from matplotlib import pyplot as plt

In [2]:
!pip uninstall keras -y
!pip install keras==2.1.6

Uninstalling Keras-2.1.6:
  Successfully uninstalled Keras-2.1.6
  Using cached https://files.pythonhosted.org/packages/54/e8/eaff7a09349ae9bd40d3ebaf028b49f5e2392c771f294910f75bb608b241/Keras-2.1.6-py2.py3-none-any.whl


In [3]:
DATADIR = "C:\\Users\\dszabo\\Desktop\\KAGGLE\\train"
CATEGORIES = ["M.Beer", "MD.Diet", "MD.Orig", "P.Cherry", "P.diet", "P.Orig", "P.Rsugar", "P.Zero"]

In [4]:
training_data = []

def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category)  
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path,img))
        training_data.append([img_array, class_num])
      except Exception as e:
        pass

In [5]:
create_training_data()

import random
random.shuffle(training_data)

x = []
y = []

for features, label in training_data:
  x.append(features)
  y.append(label)

In [6]:
'''
#saving data
import pickle

pickle_out = open("x.pickle", "wb")
pickle.dump(x, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()
'''

'\n#saving data\nimport pickle\n\npickle_out = open("x.pickle", "wb")\npickle.dump(x, pickle_out)\npickle_out.close()\n\npickle_out = open("y.pickle", "wb")\npickle.dump(y, pickle_out)\npickle_out.close()\n'

In [7]:

'''xpickle_in = open("x.pickle", "rb")
x = pickle.load(xpickle_in)

ypickle_in = open("y.pickle", "rb")
y = pickle.load(ypickle_in)
'''

'xpickle_in = open("x.pickle", "rb")\nx = pickle.load(xpickle_in)\n\nypickle_in = open("y.pickle", "rb")\ny = pickle.load(ypickle_in)\n'

In [8]:
# model exported from Deep Learning Studio, parameters have been tweaked to match Keras' current (2.1.6) API requirements 

import keras
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D #new API
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.regularizers import *

# addedd data_format = 'channels_first' for proper dimension handling

from keras import backend as bck
bck.set_image_data_format('channels_last')

# changed "Convolution2D" to "Conv2D" for Keras 2.0 API
# cahnged Convolution parameters from [Conv2D(name='Convolution2D_282',nb_row= 3,nb_filter= 32,activation= 'relu' ,nb_col= 3,data_format = 'channels_first']
# to 
# changed Dropout rate parameter "p" to "rate"
def get_model():
	aliases = {}
	Input_4 = Input(shape=(3,480, 640), name='Input_4')
	BatchNormalization_1 = BatchNormalization(name='BatchNormalization_1',axis= 1)(Input_4)
	Convolution2D_282 = Conv2D(name="Convolution2D_282", activation="relu", kernel_size=(3, 3), filters=32)(BatchNormalization_1)
	Convolution2D_283 = Conv2D(name="Convolution2D_283", activation="relu", kernel_size=(3, 3), filters=32)(Convolution2D_282)
	MaxPooling2D_39 = MaxPooling2D(name='MaxPooling2D_39',pool_size= (3, 3))(Convolution2D_283)
	Dropout_19 = Dropout(name='Dropout_19',rate= 0.25)(MaxPooling2D_39)
	Convolution2D_284 = Conv2D(name="Convolution2D_284", activation="relu", kernel_size=(3, 3), filters=64)(Dropout_19)
	Convolution2D_285 = Conv2D(name="Convolution2D_285", activation="relu", kernel_size=(3, 3), filters=64)(Convolution2D_284)
	MaxPooling2D_40 = MaxPooling2D(name='MaxPooling2D_40',pool_size= (4, 4))(Convolution2D_285)
	Dropout_20 = Dropout(name='Dropout_20',rate= 0.25)(MaxPooling2D_40)
	Convolution2D_287 = Conv2D(name="Convolution2D_287", activation="relu", kernel_size=(3, 3), filters=128)(Dropout_20)
	Convolution2D_288 = Conv2D(name="Convolution2D_288", activation="relu", kernel_size=(3, 3), filters=128)(Convolution2D_287)
	MaxPooling2D_42 = MaxPooling2D(name='MaxPooling2D_42',pool_size= (5, 5))(Convolution2D_288)
	Dropout_21 = Dropout(name='Dropout_21',rate= 0.25)(MaxPooling2D_42)
	Flatten_5 = Flatten(name='Flatten_5')(Dropout_21)
    #changed dense parameters for keras 2 api from Dense(name='Dense_15',activation= 'relu' ,output_dim= 256) to Dense(name="Dense_15", activation="relu", units=256)
	Dense_15 = Dense(256,name="Dense_15", activation="relu")(Flatten_5)
	Dense_16 = Dense(8, name="Dense_16", activation="softmax")(Dense_15)

	model = Model([Input_4],[Dense_16])
	return aliases, model


from keras.optimizers import *

def get_optimizer():
	return Adadelta()

def is_custom_loss_function():
	return False

def get_loss_function():
	return 'categorical_crossentropy'

def get_batch_size():
	return 10

def get_num_epoch():
	return 10

def get_data_config():
	return '{"kfold": 1, "numPorts": 1, "dataset": {"name": "SBC Train", "type": "private", "samples": 6619}, "mapping": {"Label": {"options": {}, "port": "OutputPort0", "shape": "", "type": "Categorical"}, "Image": {"options": {"height_shift_range": 0, "Resize": false, "horizontal_flip": false, "shear_range": 0, "rotation_range": 0, "pretrained": "None", "Width": 28, "vertical_flip": false, "Scaling": 1, "width_shift_range": 0, "Height": 28, "Augmentation": false, "Normalization": false}, "port": "InputPort0", "shape": "", "type": "Image"}}, "datasetLoadOption": "full", "shuffle": true, "samples": {"split": 3, "test": 992, "validation": 992, "training": 4633}}'

Using TensorFlow backend.


In [9]:
aliases, model = get_model()

Instructions for updating:
Colocations handled automatically by placer.


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'Convolution2D_283/convolution' (op: 'Conv2D') with input shapes: [?,1,478,32], [3,3,32,32].

In [ ]:
print(keras.__version__)

In [ ]:
model.compile(optimizer=get_optimizer(), loss=get_loss_function(), metrics=["accuracy"])

In [ ]:
#converting x and y to numpy arrays
x = np.array(x)
print(x.shape)
y = np.array(y)
print(y.shape)

In [ ]:
#reshape x to match input dimensions
x.shape = (6702, 3,480,640)
print(x.shape)

In [ ]:
'''
def split_input(x):
    return sequence[:-1], tf.reshape(sequence[1:], (-1,1))
'''

In [ ]:
#make y categorical
from keras.utils import to_categorical
print(y.shape)

y = to_categorical(y)
print(y.shape)

dataset_size = y.shape[0]
train_size = np.round(dataset_size*0.7, 0)
test_size = np.round((dataset_size-train_size)/dataset_size, 2)
validation_ratio = np.round(test_size*0.5/train_size,2)
print(train_size, test_size, validation_ratio)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size =0.3)

In [ ]:
from os.path import join
with open(join('logs\\', 'metadata.tsv'), 'w') as f:
               np.savetxt(f, y_test)

In [ ]:
from time import time
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(
    log_dir = "logs\\{}".format(time()), 
    histogram_freq = 1,
    batch_size = get_batch_size(),
    write_graph=True, 
    write_grads=True, 
    write_images=True
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train,y_train, 
    epochs=get_num_epoch(), 
    batch_size = get_batch_size(), 
    validation_data=(X_test, y_test), 
    shuffle = True, 
    callbacks=[tensorboard]
)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print('Test loss: 'score[0], 'n\Test accuracy: 'score[1])